In [1]:
import numpy as np
import CcaGrid

In [2]:
NEIGHBORHOOD_MOORE = 0
NEIGHBORHOOD_NEUMANN = 1

num_columns = 10
num_rows = 10
num_states = 3
neighborhood_range = 1
neighboorhood_type = NEIGHBORHOOD_MOORE

### Create states

create array of 100 states

In [76]:
def randomStates(rows, columns, states):
    grid = np.random.rand(num_rows*num_columns) * (states-1)
    grid = grid.round()
    return grid

In [77]:
grid = randomStates(num_rows, num_columns, num_states)

In [78]:
grid

array([2., 1., 2., 1., 1., 1., 0., 2., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 2., 0., 1., 0.,
       2., 0., 0., 0., 0., 2., 2., 1., 1., 1., 0., 0., 2., 1., 1., 1., 1.,
       1., 2., 2., 1., 2., 1., 1., 2., 0., 0., 2., 1., 1., 1., 2., 1., 0.,
       1., 0., 0., 1., 0., 2., 1., 1., 0., 2., 0., 0., 0., 0., 0., 2., 1.,
       0., 1., 1., 0., 1., 0., 2., 0., 2., 1., 1., 2., 1., 0., 2.])

### Create positions

In [6]:
def positions(rows, columns):
    return np.asarray(list(range(0, num_rows*num_columns)))

In [7]:
positions = positions(num_rows, num_columns)

In [8]:
positions

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

### Get position from coordinate

In [65]:
def coordinate_to_position(coordinate, total_cols):
    return (coordinate[0]*total_cols) + coordinate[1]

### Get coordinate from position

In [66]:
def position_to_coordinate(position, total_cols):
    return (int(position/total_cols)//1, position%total_cols)

### Create Moore neighborhood

** For cell (5, 5) **

In [10]:
current_row = 5
current_column = 5

In [11]:
def wrap_coordinate(coordinate, num_rows, num_columns):
    return (coordinate[0] % num_rows, coordinate[1] % num_columns)

In [67]:
def moore_neighborhood_positions(row, column, size, num_rows, num_columns):
    coordinates = []
    for row_offset in range(-size, size+1):
        for column_offset in range(-size, size+1):
            if (row_offset != 0) | (column_offset != 0):
                coordinates.append((row+row_offset, column+column_offset))
            
    return list(coordinate_to_position(wrap_coordinate(x, num_rows, num_columns), num_columns) for x in coordinates)

In [68]:
neighborhood_positions = moore_neighborhood_positions(current_row, current_column, 1, num_rows, num_columns)

### Create neighborhood mask

In [69]:
def neighborhood_mask(neighborhood_positions, positions):
    return np.asarray(list(x not in neighborhood_positions for x in positions))

In [70]:
neighbor_grid = neighborhood_mask(neighborhood_positions, positions)

In [18]:
np.asarray(neighbor_grid).reshape(10, 10)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True, False, False, False,  True,  True,
         True],
       [ True,  True,  True,  True, False,  True, False,  True,  True,
         True],
       [ True,  True,  True,  True, False, False, False,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])

### Use mask to query neighborhood

In [19]:
neighbor_states = np.ma.masked_array(grid, neighbor_grid)

In [20]:
neighbor_states

masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, 1.0, 1.0, 1.0, --, --, --, --, --, --, --, 1.0,
                   --, 1.0, --, --, --, --, --, --, --, 2.0, 2.0, 1.0, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --],
             mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True, False, False, Fal

In [21]:
neighbor_states.compressed()

array([1., 1., 1., 1., 1., 2., 2., 1.])

In [22]:
(neighbor_states.compressed() == 1).sum()

6

In [23]:
(neighbor_states.compressed() == 2).sum()

2

### Create all neighborhood masks

In [45]:
def moore_neighborhood_size(size):
    return ((2*size) + 1) ** 2 - 1

In [72]:
position_neighborhood_positions = np.empty((num_rows*num_columns, moore_neighborhood_size(1)));
for x in positions:
    coordinate = position_to_coordinate(x, num_columns)
    position_neighborhood_positions[x] = moore_neighborhood_positions(coordinate[0], coordinate[1], 1, num_rows, num_columns)

In [73]:
position_neighborhood_positions

array([[99., 90., 91.,  9.,  1., 19., 10., 11.],
       [90., 91., 92.,  0.,  2., 10., 11., 12.],
       [91., 92., 93.,  1.,  3., 11., 12., 13.],
       [92., 93., 94.,  2.,  4., 12., 13., 14.],
       [93., 94., 95.,  3.,  5., 13., 14., 15.],
       [94., 95., 96.,  4.,  6., 14., 15., 16.],
       [95., 96., 97.,  5.,  7., 15., 16., 17.],
       [96., 97., 98.,  6.,  8., 16., 17., 18.],
       [97., 98., 99.,  7.,  9., 17., 18., 19.],
       [98., 99., 90.,  8.,  0., 18., 19., 10.],
       [ 9.,  0.,  1., 19., 11., 29., 20., 21.],
       [ 0.,  1.,  2., 10., 12., 20., 21., 22.],
       [ 1.,  2.,  3., 11., 13., 21., 22., 23.],
       [ 2.,  3.,  4., 12., 14., 22., 23., 24.],
       [ 3.,  4.,  5., 13., 15., 23., 24., 25.],
       [ 4.,  5.,  6., 14., 16., 24., 25., 26.],
       [ 5.,  6.,  7., 15., 17., 25., 26., 27.],
       [ 6.,  7.,  8., 16., 18., 26., 27., 28.],
       [ 7.,  8.,  9., 17., 19., 27., 28., 29.],
       [ 8.,  9.,  0., 18., 10., 28., 29., 20.],
       [19., 10., 11

### Next steps
* Work with (views on) matrices
* Only work in 1 dimension until it is time to visualize the grid